In [1]:
# #!/bin/bash

# # Assume you are in my_project directory

# newdir="language_model"
# if [ ! -d "$newdir" ]; then
#     mkdir "$newdir"
# fi

# cd "$newdir"

# lmdir="lm_1b"
# if [ ! -d "$lmdir" ]; then
#     mkdir "$lmdir"
# fi

# wget https://raw.githubusercontent.com/abaybektursun/nlp_mc/master/network/language_model/lm_1b/data_utils.py -O "$lmdir/data_utils.py";

# datdir="data"
# if [ ! -d "$datdir" ]; then
#     mkdir "$datdir"
# fi

# # Download the Weights. Takes long time
# url_base="http://download.tensorflow.org/models/LM_LSTM_CNN/"
# shards="all_shards-2016-09-10/"
# model_urls=(ckpt-base ckpt-char-embedding ckpt-lstm ckpt-softmax0 ckpt-softmax1 ckpt-softmax2 ckpt-softmax3 ckpt-softmax4 ckpt-softmax5 ckpt-softmax6 ckpt-softmax7 ckpt-softmax8)
# ITER=1; NFILES=${#model_urls[@]}
# for a_url in ${model_urls[*]} 
# do
#     echo "-------------------------------------------------------------------------------------------------------"
#     echo "	 Downloading file $ITER out of $NFILES"
#     echo "-------------------------------------------------------------------------------------------------------"
    
#     if [ -f "$datdir/$a_url" ]; then
#         echo "$a_url is already downloaded, skipping"
#     else
#         wget "$url_base$shards$a_url" -O "$datdir/$a_url" -q --show-progress
#     fi

#     let ITER=${ITER}+1
# done

# # Gotta do separate because this ones have different bases
# wget $url_base"test/news.en.heldout-00000-of-00050" -O "$datdir/news.en.heldout-00000-of-00050" -q --show-progress
# wget $url_base"graph-2016-09-10.pbtxt" -O "$datdir/graph-2016-09-10.pbtxt" -q --show-progress
# wget $url_base"vocab-2016-09-10.txt" -O "$datdir/vocab-2016-09-10.txt" -q --show-progress

# echo "data/" > ".gitignore"
!touch setup.sh


In [7]:
!bash setup.sh

--2022-05-01 10:02:30--  https://raw.githubusercontent.com/abaybektursun/nlp_mc/master/network/language_model/lm_1b/data_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8407 (8.2K) [text/plain]
Saving to: ‘lm_1b/data_utils.py’

lm_1b/data_utils.py 100%[===================>]   8.21K  --.-KB/s    in 0s      

2022-05-01 10:02:30 (70.1 MB/s) - ‘lm_1b/data_utils.py’ saved [8407/8407]

-------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------
ckpt-base is already downloaded, skipping
-------------------------------------------------------------------------------------------------------
---------------------

In [8]:
!pip3 install tensorflow==1.15.5

In [1]:
import os
import sys
import numpy as np
import tensorflow as tf

# From lm_1b
import language_model.lm_1b.data_utils as data_utils

from six.moves       import xrange
from google.protobuf import text_format

#-------------------------------------------------------------------------------
# Adopted from lm_1b_eval.py
def LoadModel(gd_file, ckpt_file):
    """Load the model from GraphDef and Checkpoint.
    Args: gd_file: GraphDef proto text file. ckpt_file: TensorFlow Checkpoint file.
    Returns: TensorFlow session and tensors dict."""
    with tf.Graph().as_default():
        #class FastGFile: File I/O wrappers without thread locking.
        with tf.gfile.FastGFile(gd_file, 'r') as f:
            # Py 2: s = f.read().decode()
            s = f.read()
            # Serialized version of Graph
            gd = tf.GraphDef()
            # Merges an ASCII representation of a protocol message into a message.
            text_format.Merge(s, gd)

        tf.logging.info('Recovering Graph %s', gd_file)

        t = {}
        [t['states_init'], t['lstm/lstm_0/control_dependency'],
         t['lstm/lstm_1/control_dependency'], t['softmax_out'], t['class_ids_out'],
         t['class_weights_out'], t['log_perplexity_out'], t['inputs_in'],
         t['targets_in'], t['target_weights_in'], t['char_inputs_in'],
         t['all_embs'], t['softmax_weights'], t['global_step']
        ] = tf.import_graph_def(gd, {}, ['states_init',
                                         'lstm/lstm_0/control_dependency:0',
                                         'lstm/lstm_1/control_dependency:0',
                                         'softmax_out:0',
                                         'class_ids_out:0',
                                         'class_weights_out:0',
                                         'log_perplexity_out:0',
                                         'inputs_in:0',
                                         'targets_in:0',
                                         'target_weights_in:0',
                                         'char_inputs_in:0',
                                         'all_embs_out:0',
                                         'Reshape_3:0',
                                         'global_step:0'], name='')


        sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
        sess.run('save/restore_all', {'save/Const:0': ckpt_file})
        sess.run(t['states_init'])

    return sess, t


In [2]:

BATCH_SIZE = 1
NUM_TIMESTEPS = 1
MAX_WORD_LEN = 50

# File Paths
vocab_file = "language_model/data/vocab-2016-09-10.txt"
save_dir   = "language_model/output"
pbtxt      = "language_model/data/graph-2016-09-10.pbtxt"
ckpt       = "language_model/data/ckpt-*"

#Vocabulary containing character-level information.
vocab = data_utils.CharsVocabulary(vocab_file, MAX_WORD_LEN)

targets  = np.zeros([BATCH_SIZE, NUM_TIMESTEPS], np.int32)
weights  = np.ones([BATCH_SIZE, NUM_TIMESTEPS], np.float32)
inputs   = np.zeros([BATCH_SIZE, NUM_TIMESTEPS], np.int32)
char_ids_inputs = np.zeros([BATCH_SIZE, NUM_TIMESTEPS, vocab.max_word_length], np.int32)

# Recovers the model from protobuf
sess, t = LoadModel(pbtxt, ckpt)


Instructions for updating:
Use tf.gfile.GFile.
INFO:tensorflow:Recovering Graph language_model/data/graph-2016-09-10.pbtxt


In [3]:
def forward(sentence):
    # Tokenize characters and words
    word_ids = [vocab.word_to_id(w) for w in sentence.split()]
    char_ids = [vocab.word_to_char_ids(w) for w in sentence.split()]

    if sentence.find('<S>') != 0:
        sentence = '<S> ' + sentence

    for i in xrange(len(word_ids)):
        inputs[0, 0] = word_ids[i]
        char_ids_inputs[0, 0, :] = char_ids[i]

        # Add 'lstm/lstm_0/control_dependency' if you want to dump previous layer
        # LSTM.
        lstm_emb = sess.run(t['lstm/lstm_1/control_dependency'],
                        feed_dict={t['char_inputs_in']: char_ids_inputs,
                                   t['inputs_in']: inputs,
                                   t['targets_in']: targets,
                                   t['target_weights_in']: weights})
    return lstm_emb

### Getting embeddings

In [ ]:
final_embeds = []

cosm_data = [
        # "If you're having trouble orgasming in a standard partner-in-between-your-legs position, switch it up and mount his face, being careful not to apply too much pressure",
        "Here's a unique way to give his frenulum — the tiny bump on the underside of his penis where the shaft meets the tip—some special attention: Place the tip of your finger on it, then take his shaft (along with your finger) into your mouth",
        # "You hundred percent do not need to be on your knees to give your partner oral sex",
        ]


for text in cosm_data:
    # final_embeds.append(forward(text).reshape(-1))    
    # word_ids = [vocab.word_to_id(w) for w in text.split()]
    char_ids = [vocab.word_to_char_ids(w) for w in text.split()]


In [ ]:
text.split()[7]

'frenulum—the'

In [ ]:
vocab.word_to_char_ids("—") ## There are some characters that are problematic

array([   2, 8212,    3,    4,    4,    4,    4,    4,    4,    4,    4,
          4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
          4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
          4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
          4,    4,    4,    4,    4,    4], dtype=int32)

In [ ]:
import re
s = "string. With. Punctuation?"
s = re.sub(r'[^\w\s]','',s)
s

'string With Punctuation'

In [5]:
import numpy as np

subj_data = np.load("stimuli_words.npy",allow_pickle=True)

In [6]:
subj_data

array(['Harry', 'had', 'never', ..., 'thirteen', 'was.', '+'],
      dtype='<U14')

In [14]:
import re

subj_data_formatted = list()
temp = list()
for w in subj_data:
  if w!='+':
    temp.append(w)
  else:
    temp_word = " ".join(temp)
    s=temp_word
    s = re.sub(r'[^\w\s]','',temp_word)
    temp = list()
    if s == '':
      continue

    subj_data_formatted.append(s)
subj_data_formatted

['Harry had never believed he would meet a boy he hated more than Dudley but that was before he met Draco Malfoy Still firstyear Gryffindors only had Potions with the Slytherins so they didnt have to put up with Malfoy much Or at least they didnt until they spotted a notice pinned up in the Gryffindor common room that made them all groan Flying lessons would be starting on Thursday  and Gryffindor and Slytherin would be learning together',
 'Typical said Harry darkly Just what I always wanted To make a fool of myself on a broomstick in front of Malfoy',
 'He had been looking forward to learning to fly more than anything else You dont know that youll make a fool of yourself said Ron reasonably Anyway I know Malfoys always going on about how good he is at Quidditch but I bet thats all talk',
 'Malfoy certainly did talk about flying a lot He complained loudly about first years never getting on the House Quidditch teams and told long boastful stories that always seemed to end with him narr

In [ ]:
from tqdm import tqdm


subj_embeds= list()
for text in tqdm(subj_data_formatted):
    subj_embeds.append(forward(text).reshape(-1))


  0%|          | 0/198 [00:00<?, ?it/s]

In [ ]:
subj_embeds = np.array(subj_embeds)
subj_embeds

array([[ 0.14065225,  0.00736293, -0.4607407 , ...,  0.02256148,
        -0.209079  , -0.5298386 ],
       [-0.05774261, -0.12987113, -0.02990648, ...,  0.26181778,
        -0.4751227 , -0.23012371],
       [ 0.0251148 , -0.4531601 , -0.67646086, ...,  0.27137366,
        -0.34036297,  0.0133964 ],
       ...,
       [-0.46694604,  0.01720074, -0.59188956, ..., -0.42356327,
        -0.6471288 , -0.2694936 ],
       [-0.06491936,  0.00350159, -0.33017522, ..., -0.2221539 ,
        -0.39522478, -0.8648159 ],
       [ 1.0308399 ,  0.2867789 ,  0.5290635 , ...,  0.6679201 ,
         0.879962  , -0.4553225 ]], dtype=float32)

In [ ]:
subj_embeds.dump("/content/drive/MyDrive/colabdata/LSTM_Embeddings.npy")

In [ ]:
np.load("/content/drive/MyDrive/colabdata/LSTM_Embeddings.npy", allow_pickle=True)

array([[ 0.14065225,  0.00736293, -0.4607407 , ...,  0.02256148,
        -0.209079  , -0.5298386 ],
       [-0.05774261, -0.12987113, -0.02990648, ...,  0.26181778,
        -0.4751227 , -0.23012371],
       [ 0.0251148 , -0.4531601 , -0.67646086, ...,  0.27137366,
        -0.34036297,  0.0133964 ],
       ...,
       [-0.46694604,  0.01720074, -0.59188956, ..., -0.42356327,
        -0.6471288 , -0.2694936 ],
       [-0.06491936,  0.00350159, -0.33017522, ..., -0.2221539 ,
        -0.39522478, -0.8648159 ],
       [ 1.0308399 ,  0.2867789 ,  0.5290635 , ...,  0.6679201 ,
         0.879962  , -0.4553225 ]], dtype=float32)